In [ ]:
library(CellChat)
library(Matrix)
library(patchwork)

In [ ]:
expr_data <- readRDS("./negative_sc_adata_expr.rds")
log_expr_data <- log1p(expr_data)
meta <- read.csv('./negative_sc_adata_meta.csv', row.names = 1, check.names=FALSE)
cellchat_pos <- createCellChat(object = t(log_expr_data), meta = meta, group.by = 'Type')
CellChatDB <- CellChatDB.human
cellchat_pos@DB <- CellChatDB
cellchat_pos <- subsetData(cellchat_pos)

options(future.globals.maxSize = 10 * 1024^3)
future::plan("multisession", workers = 8)

cellchat <- identifyOverExpressedGenes(cellchat_pos) # Use fast mode
cellchat <- identifyOverExpressedInteractions(cellchat)
cellchat <- computeCommunProb(cellchat, type='triMean', population.size = TRUE)
cellchat <- filterCommunication(cellchat, min.cells = 10)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)

saveRDS(cellchat,"./negative_sc_cellchat.rds")


In [ ]:
expr_data <- readRDS("./positive_sc_adata_expr.rds")
log_expr_data <- log1p(expr_data)
meta <- read.csv('./positive_sc_adata_meta.csv', row.names = 1, check.names=FALSE)
cellchat_pos <- createCellChat(object = t(log_expr_data), meta = meta, group.by = 'Type')
CellChatDB <- CellChatDB.human
cellchat_pos@DB <- CellChatDB
cellchat_pos <- subsetData(cellchat_pos)

options(future.globals.maxSize = 10 * 1024^3)
future::plan("multisession", workers = 8)

cellchat <- identifyOverExpressedGenes(cellchat_pos) # Use fast mode
cellchat <- identifyOverExpressedInteractions(cellchat)
cellchat <- computeCommunProb(cellchat, type='triMean', population.size = TRUE)
cellchat <- filterCommunication(cellchat, min.cells = 10)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)

saveRDS(cellchat,"./positive_sc_cellchat.rds")


In [ ]:
cellchat.pos <- readRDS("./positive_sc_cellchat.rds")
cellchat.neg <- readRDS("./negative_sc_cellchat.rds")
object.list <- list(pos = cellchat.pos, neg = cellchat.neg)
cellchat <- mergeCellChat(object.list, add.names = names(object.list))


In [ ]:
### FigS3j

source.types <- c("Epithelial")   # 发出信号方
target.types <- c("Fibroblast","Myofibroblast")   # 接收信号方

gg1 <- rankNet(
  cellchat,
  mode = "comparison",
  stacked = TRUE,
  do.stat = TRUE,
  sources.use = source.types,
  targets.use = target.types
)


source.types <- c("Epithelial")   # 发出信号方
target.types <- c("T-cell")   # 接收信号方

gg2 <- rankNet(
  cellchat,
  mode = "comparison",
  stacked = TRUE,
  do.stat = TRUE,
  sources.use = source.types,
  targets.use = target.types
)


In [ ]:
### Fig2j

pathways.show <- c("TGFb") 
weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
par(mfrow = c(1,2), xpd=TRUE)
for (i in 1:length(object.list)) {
  netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", edge.weight.max = weight.max[1], edge.width.max = 5, signaling.name = paste(pathways.show, names(object.list)[i]))
}


In [ ]:
### FigS2k

pathways.show <- c("MHC-I") 
weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) # control the edge weights across different datasets
par(mfrow = c(1,2), xpd=TRUE)
for (i in 1:length(object.list)) {
  netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", edge.weight.max = weight.max[1], edge.width.max = 5, signaling.name = paste(pathways.show, names(object.list)[i]))
}